In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from operator import add

sc = SparkContext(appName="task2")


lines_2 = sc.textFile("record.txt") #read criminal list file
criminal_record=lines_2.flatMap(lambda x: x.split(" ")).collect()


lines = sc.textFile("relationship.txt") #read relationship list file

def record(t):
    if t in criminal_record :return [1,1]
    else: return [1,0]

result=lines.map(lambda x: x.split(" "))  ## ['name1','name2']


####collection: number of [friends & criminous friends]
relation=result.map(lambda word: (word[0],record(word[1]))).reduceByKey(lambda x,y: [x[0]+y[0],x[1]+y[1]])
relation.collect()

[('bob', [3, 1]),
 ('tom', [3, 1]),
 ('mike', [4, 3]),
 ('lily', [2, 1]),
 ('tim', [1, 0]),
 ('zed', [1, 0])]

In [2]:
####percentage of criminal in friends list
percent = relation.mapValues(lambda s: s[1]/s[0])
percent.collect()

[('bob', 0.3333333333333333),
 ('tom', 0.3333333333333333),
 ('mike', 0.75),
 ('lily', 0.5),
 ('tim', 0.0),
 ('zed', 0.0)]

In [3]:
####filter the person who is in risk
def risk(p):
    if p[1]>0.5 :return True
    else: return False

risk_list = percent.filter(risk)
risk_list.collect()

[('mike', 0.75)]